# Gathering and Investigating Materials Project Data

This notebooks will show how you can use `requests` and `pandas` so gather and explore your data. Often times you will need to suply your data by other methods.

In [3]:
import requests
import pandas as pd

base_url = 'https://materialsproject.org/rest/v2/'

# Getting Materials Project Api Key

This [link](https://www.materialsproject.org/open) details the steps necissary. 

1. Visit [dashboard](https://materialsproject.org/dashboard) you may need to login
2. Generate API key if it has not already been generated and set `API_KEY` to this value.

The subprocess method is a way that I store my passwords on my computer and will not work for you.

Afterwards in the next cell we will test that our API key works. 

This is done by performing a `GET` or `POST` request to `https://www.materialsproject.org/rest/v1/api_check`.

In [4]:
import subprocess
API_KEY = subprocess.check_output('gopass www/materialsproject.com apikey'.split()).decode('utf-8')
# API_KEY = "<apikey-here>"

session = requests.Session()
session.headers.update({'X-API-KEY': API_KEY})

In [5]:
# for some reason the v2 API does not include an API check method??
response = session.get(f'https://www.materialsproject.org/rest/v1/api_check')
data = response.json()
print(data)

if not data['api_key_valid']:
    raise ValueError('You are not authenticated!')

{'valid_response': True, 'api_key_valid': True}


# Materials Project API

The materials project provides a RESTfull API for getting material properties which is detailed [here](https://www.materialsproject.org/docs/api#materials_.28calculated_materials_data.29).

If you have followed the steps above you should be ready to parse materials project data.

A RESTfull API is a nice way to expose data over the web. While they provide convenient methods for getting each individual material property they have a limit of 500 queries per day so we need to be efficient in our queries. To do this we will use the `npquery` to get properties in batch.

Lets start by getting a list of materials that are compossed of the following elements `Fe`, `Ti`, `O`, `C`, `N`, `He`. This does not affect your API limit

In [82]:
def get_materials(elements):
    elements_str = '-'.join(elements)
    response = session.get(f'{base_url}/materials/{elements_str}/mids')
    data = response.json()
    print(f'Found {len(data["response"])} Materials in the Materials Project with the elements: {elements}')
    return data['response']

def get_material_vasp_properties(mid, piezoelectric=False, dielelectric=False):
    response = session.get(f'{base_url}/materials/{mid}/vasp/')
    material_data = response.json()['response'][0]
    
    if piezoelectric:
        response = session.get(f'{base_url}/materials/{mid}/vasp/piezo')
        data = response.json()
        if not data['valid_response']:
            material_data['piezoelectric'] = None
        else:
            material_data['piezoelectric'] = data['response']
        
    if dielelectric:
        response = session.get(f'{base_url}/materials/{mid}/vasp/diel')
        data = response.json()
        if not data['valid_response']:
            material_data['dielelectric'] = None
        else:
            material_data['dielelectric'] = data['response']
    
    return material_data

In [83]:
material_ids = get_materials(['Fe', 'O', 'Ni', 'He', 'Ni', 'Cu'])

Found 253 Materials in the Materials Project with the elements: ['Fe', 'O', 'Ni', 'He', 'Ni', 'Cu']


In [84]:
material_id = material_ids[20]

In [85]:
material_id

'mp-705571'

# Basic VASP properties

Includes:

 - `energy`, `energy_per_atom`, `volume`, `formation_energy_per_atom`, `nsites`, `unit_cell_formula`, `pretty_formula`, `e_above_hull`, `spacegroup`, `icsd_ids`, `cif`, 

 - properties: `band_gap`, `density`, `energry`, `energy_per_atom`, `formation_energy_per_atom`, `elascticity`, `total_magnetization`
 
But some properties are still not included:
  
 -  `piezo`, `diel`

In [86]:
# MgO

material_id = 'mp-1265'

# Na2O

material_id = 'mp-776952'

In [87]:
get_material_vasp_properties(material_id, piezoelectric=True, dielelectric=True)

{'energy': -32.94803928,
 'energy_per_atom': -3.6608932533333336,
 'volume': 123.22298842543988,
 'formation_energy_per_atom': -1.3747529237500007,
 'nsites': 9,
 'unit_cell_formula': {'Na': 6.0, 'O': 3.0},
 'pretty_formula': 'Na2O',
 'is_hubbard': False,
 'elements': ['Na', 'O'],
 'nelements': 2,
 'e_above_hull': 0.07964257999999935,
 'hubbards': {},
 'is_compatible': True,
 'spacegroup': {'source': 'spglib',
  'symbol': 'P3',
  'number': 143,
  'point_group': '3',
  'crystal_system': 'trigonal',
  'hall': 'P 3'},
 'task_ids': ['mp-776952',
  'mp-788131',
  'mp-794064',
  'mp-794687',
  'mp-1092873',
  'mp-1096781'],
 'band_gap': 1.5089000000000001,
 'density': 2.5056634752592992,
 'icsd_id': None,
 'icsd_ids': [],
 'cif': "# generated using pymatgen\ndata_Na2O\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   6.19425281\n_cell_length_b   7.21866638\n_cell_length_c   7.21895917\n_cell_angle_alpha   95.98067380\n_cell_angle_beta   64.59330061\n_cell_angle_gamma   149.08497211\n